In [ ]:
import pandas as pd

# Loading dataset
ind_df = pd.read_csv('data/indigenous_collection.csv', index_col='id')
print(f'Dataframe columns: \n{ind_df.columns}')

## Transforming Images

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Visualizing first image
first_image = ind_df['image_path'].loc[55663]
plt.imshow(cv2.imread(first_image))

In [ ]:
from tqdm import tqdm
import scipy as scp
import numpy as np

# Extracting height and width information from entire dataset for later processing
dimensions = {'height': [], 'width': []}
for index, value in tqdm(ind_df['image_path'].dropna().items(), total=len(ind_df['image_path'].dropna()), desc='Reading images'):
    try:
        image = cv2.imread(value)
        dimensions['height'].append(image.shape[0])
        dimensions['width'].append(image.shape[1])
    except:
        print('Corrupted image: {}'.format(value))

print('''\nStatistics on height of images: \n{}
quantiles (0.25, 0.50, 0.75, 0.90) = {}, {}, {}, {}'''.format(scp.stats.describe(dimensions['height']), \
                                                              np.quantile(dimensions['height'], 0.25), \
                                                              np.quantile(dimensions['height'], 0.50), \
                                                              np.quantile(dimensions['height'], 0.75), \
                                                              np.quantile(dimensions['height'], 0.90)))
print('''\nStatistics on width of images: \n{}
quantiles (0.25, 0.50, 0.75, 0.90) = {}, {}, {}, {}'''.format(scp.stats.describe(dimensions['width']), \
                                                              np.quantile(dimensions['width'], 0.25), \
                                                              np.quantile(dimensions['width'], 0.50), \
                                                              np.quantile(dimensions['width'], 0.75), \
                                                              np.quantile(dimensions['width'], 0.90)))

In [ ]:
from PIL import Image
import torch
from torchvision import transforms
from transformers import pipeline, AutoModelForImageSegmentation

# UPDATE REQUIREMENTS IN WITH: scikit-image, timm, kornia

# Removing backgroung
seg_pipe = pipeline('image-segmentation', model='briaai/RMBG-1.4', trust_remote_code=True)
pillow_image = seg_pipe(first_image)
pillow_image.save("first_image_br_r.png")


model = AutoModelForImageSegmentation.from_pretrained('briaai/RMBG-2.0', trust_remote_code=True)
torch.set_float32_matmul_precision(['high', 'highest'][0])
model.to('cuda')
model.eval()

image_size = (np.quantile(dimensions['width'], 0.75), np.quantile(dimensions['height'], 0.75))
transform_image = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

image = Image.open(first_image)
input_images = transform_image(image).unsqueeze(0).to('cuda')

with torch.no_grad():
    preds = model(input_images)[-1].sigmoid().cpu()
pred = preds[0].squeeze()
pred_pil = transforms.ToPILImage()(pred)
mask = pred_pil.resize(image.size)
image.putalpha(mask)
image.save("first_image_br_r.png")